In [1]:
from typing import TypedDict, List, Union
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
import os
import subprocess
import tempfile

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***State***

In [2]:
class State(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

***LLM = GPT-4o-mini***

In [3]:
model_name = "openai/gpt-4o-mini"
api_key = os.getenv("API_KEY")
llm = ChatOpenAI(
    model=model_name, 
    openai_api_base = "https://openrouter.ai/api/v1", 
    openai_api_key = api_key, 
    temperature = 0.7
)

***FIRST NODE : ANALYZER***

In [4]:
analyzer_prompt = """
You are a problem-solving assistant for programming Olympiad exercises. 
You will be given a problem description.

Task:
1. Read the problem carefully.
2. Break it down into a step-by-step plan to solve it.
3. List each step clearly, numbered as Step 1, Step 2, etc.
4. Be precise, concise, and focus on logic, not code.
5. Highlight any important constraints, edge cases, or special conditions.

Output format:
Step 1: ...
Step 2: ...
Step 3: ...
...
"""

def analyzer(state: State) -> State:
    last_human_msg = ''
    # reversed to choose the last one
    for msg in reversed(state['messages']):
        if isinstance(msg, HumanMessage):
            last_human_msg = msg.content
            break

    # prompt = [
    #     AIMessage(content=analyzer_prompt), 
    #     HumanMessage(content=last_human_msg)
    # ]
    
    # i don't know why the base messages don't go with 'llm.invoke' i guess i has some libraries conflicts, anyways let's use :
    prompt = analyzer_prompt + "\n\nProblem:\n" + last_human_msg
    results = llm.invoke(prompt).content
    print("analyzer results : \n", results)
    state['messages'].append(AIMessage(content=results))
    return state

***2nd NODE: Code Generator***

In [5]:
code_prompt = """
You are a python code-generating assistant. 
You have a step-by-step plan for solving a programming problem. 
Convert the steps into working Python code. 
Do not explain anything, output only the code, if you want to write a note use the comment. 
Make sure to handle edge cases mentioned in the steps.
Do NOT include any text, explanations, or markdown formatting, just the raw code. 
"""

def CodeGenerator(state: State) -> State :
    last_system_message = ''
    for msg in reversed(state['messages']):
        if isinstance(msg , AIMessage):
            last_system_message = msg.content
            break

    prompt = code_prompt + "\n" + last_system_message
    
    results = llm.invoke(prompt).content
    print("code generator : \n", results)
    state['messages'].append(AIMessage(content=results))
    return state

***3rd Node : Excuter / verifier***

In [6]:
excuter_prompt = """
You are a problem-solving assistant for programming projects. 
You will be given Python code that was generated.

Task:
1. Execute the code and observe the output.
2. Provide a short, concise feedback paragraph (≤100 words):
   - Mention if it executed correctly.
   - Highlight any errors or ex ceptions.
   - Mention obvious edge cases that might fail.
Important: If the code raises ModuleNotFoundError, DO NOT call it an error. Just warn the user to install the missing package.
"""


def Excuter(state: State) -> State:
    code_to_run = ''
    for msg in reversed(state['messages']):
        if isinstance(msg, AIMessage):
            code_to_run = msg.content
            break

    lines = code_to_run.split("\n")
    if not lines[0].startswith(("import", "def", "class", "#")):
        lines = lines[1:]    
    clean_code = "\n".join(lines)    
    
    # create a temporary file '.py'
    with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as tmp_file:
        tmp_file.write(clean_code)
        tmp_file_path = tmp_file.name

    try:
        result = subprocess.run(
            ["python", tmp_file_path],
            capture_output=True,   
            text=True,             
            timeout=None         
        )

        if result.returncode == 0:
            code_output = result.stdout.strip() or "Code executed successfully"
        else:
            code_output = f"Error:\n{result.stderr.strip()}"

    except subprocess.TimeoutExpired:
        code_output = "Error: Code execution timed out"
    finally:
        os.remove(tmp_file_path)

    prompt = excuter_prompt + "\ncode:\n" + code_to_run + "\n\noutput:\n" + code_output
    feedback = llm.invoke(prompt).content
    state['messages'].append(AIMessage(content=feedback))
    print("Execution feedback:\n", feedback)

    return state

In [ ]:
state = {
    "messages": [
        HumanMessage(content="I need a Python code that draws a red heart using turtle library")
    ]
}

In [11]:
# analyzing 
state = analyzer(state=state)

analyzer results : 
 Step 1: **Understand the Turtle Graphics Library**  
Familiarize yourself with the turtle graphics library in Python. It is used for drawing shapes and is based on a cursor (the turtle) that moves around the screen.

Step 2: **Define the Heart Shape**  
A heart shape can be represented mathematically or can be drawn using specific turtle commands. The basic outline of a heart can be created using curves and lines.

Step 3: **Set Up the Turtle Environment**  
Initialize the turtle environment by importing the turtle module and setting up the screen parameters (like background color and turtle speed).

Step 4: **Move the Turtle to Starting Position**  
Position the turtle at an appropriate starting point on the screen. This usually involves lifting the pen, moving to the desired coordinates, and putting the pen down.

Step 5: **Draw the Heart Shape**  
Use a series of turtle commands to draw curves and lines that form a heart. This typically involves using the `circl

In [ ]:

state = CodeGenerator(state=state)

code generator : 
 import turtle

def draw_heart():
    # Step 3: Set Up the Turtle Environment
    screen = turtle.Screen()
    screen.bgcolor("white")
    heart_turtle = turtle.Turtle()
    heart_turtle.speed(1)

    # Step 4: Move the Turtle to Starting Position
    heart_turtle.penup()
    heart_turtle.goto(0, -200)
    heart_turtle.pendown()

    # Step 5: Draw the Heart Shape
    heart_turtle.begin_fill()
    heart_turtle.color("red")
    
    heart_turtle.left(140)
    heart_turtle.forward(224)
    heart_turtle.circle(-112, 200)
    heart_turtle.left(120)
    heart_turtle.circle(-112, 200)
    heart_turtle.forward(224)
    
    heart_turtle.end_fill()

    # Step 7: Complete the Drawing
    heart_turtle.hideturtle()
    turtle.done()

draw_heart()


In [13]:
state = Excuter(state=state)

Execution feedback:
 The code executed successfully without any errors. It correctly draws a heart shape using the Turtle graphics library. However, ensure you have the Turtle module installed in your Python environment, as it may not be available in some setups. An edge case to consider is the performance on very small screens or when the window size is limited, as the heart might not be fully visible.


In [19]:
parts = state['messages'][-1].content.split('.')
for part in parts:
    print(part)

The code executed successfully without any errors
 It correctly draws a heart shape using the Turtle graphics library
 However, ensure you have the Turtle module installed in your Python environment, as it may not be available in some setups
 An edge case to consider is the performance on very small screens or when the window size is limited, as the heart might not be fully visible

